# Calulcates the Predictions Accuracy

In [14]:
import pandas as pd

year = 2025
week = 4

In [15]:
actual = pd.read_csv(f'cfbd_{year}_{week}_games.csv')
predictions = pd.read_csv(f'season{year}_week{week}_predictions.csv')

In [16]:
actual.head()

,season,week,homeId,homeTeam,awayTeam,awayId,homePoints,awayPoints,home_year_x,home_elo,...,away_year_y,away_epa,away_epaAllowed,away_successRate,away_successRateAllowed,away_explosiveness,away_explosivenessAllowed,away_year,away_rank,away_points
0,2025,4,2429,Charlotte,Rice,242,17,28,2025,1183,...,2025,0.124197,0.163690,0.401121,0.382108,0.925376,1.025857,2025,100,146.64
1,2025,4,197,Oklahoma State,Tulsa,202,12,19,2025,1176,...,2025,0.118857,0.162311,0.396996,0.436012,1.034668,0.995825,2025,109,137.85
2,2025,4,164,Rutgers,Iowa,2294,28,38,2025,1583,...,2025,0.179588,0.138611,0.437079,0.412342,0.923832,0.941190,2025,38,208.27
3,2025,4,193,Miami (OH),UNLV,2439,38,41,2025,1423,...,2025,0.203385,0.168189,0.430603,0.454930,0.990813,0.992641,2025,95,154.36
4,2025,4,97,Louisville,Bowling Green,189,40,17,2025,1755,...,2025,0.151492,0.211114,0.398707,0.448116,1.014863,0.992471,2025,94,154.51


In [17]:
predictions.head()

,season,week,awayTeam,homeTeam,pred_homePoints,pred_awayPoints,pred_pointDiff
0,2025,4,Rice,Charlotte,12.62,18.22,-5.60
1,2025,4,Tulsa,Oklahoma State,26.58,14.97,11.61
2,2025,4,Iowa,Rutgers,28.86,24.20,4.66
3,2025,4,SMU,TCU,37.07,28.85,8.22
4,2025,4,Maryland,Wisconsin,25.40,13.80,11.60


In [18]:
merged = pd.merge(
    actual,
    predictions,
    on=['season', 'week', 'homeTeam', 'awayTeam'],
    suffixes=('_actual', '_pred')
)


In [19]:
merged.columns	

Index(['season', 'week', 'homeId', 'homeTeam', 'awayTeam', 'awayId',
       'homePoints', 'awayPoints', 'home_year_x', 'home_elo', 'home_fpi',
       'home_rating', 'home_offense', 'home_defense', 'home_year_y',
       'home_epa', 'home_epaAllowed', 'home_successRate',
       'home_successRateAllowed', 'home_explosiveness',
       'home_explosivenessAllowed', 'home_year', 'home_rank', 'home_points',
       'away_year_x', 'away_elo', 'away_fpi', 'away_rating', 'away_offense',
       'away_defense', 'away_year_y', 'away_epa', 'away_epaAllowed',
       'away_successRate', 'away_successRateAllowed', 'away_explosiveness',
       'away_explosivenessAllowed', 'away_year', 'away_rank', 'away_points',
       'pred_homePoints', 'pred_awayPoints', 'pred_pointDiff'],
      dtype='object')

In [22]:
# Merge on matchup columns (adjust if your column names differ)
merged = pd.merge(
    actual,
    predictions,
    on=['season', 'week', 'homeTeam', 'awayTeam'],
    suffixes=('_actual', '_pred')
)

# Calculate absolute error for home and away scores
merged['home_error'] = (merged['homePoints'] - merged['pred_homePoints']).abs()
merged['away_error'] = (merged['awayPoints'] - merged['pred_awayPoints']).abs()

# Calculate mean absolute error
mae_home = merged['home_error'].mean()
mae_away = merged['away_error'].mean()

# Determine if the predicted winner matches the actual winner
merged['actual_winner'] = merged.apply(
    lambda row: 'home' if row['homePoints'] > row['awayPoints']
    else ('away' if row['homePoints'] < row['awayPoints'] else 'tie'),
    axis=1
)
merged['pred_winner'] = merged.apply(
    lambda row: 'home' if row['pred_homePoints'] > row['pred_awayPoints']
    else ('away' if row['pred_homePoints'] < row['pred_awayPoints'] else 'tie'),
    axis=1
)
merged['correct_prediction'] = merged['actual_winner'] == merged['pred_winner']

# Calculate accuracy
accuracy = merged['correct_prediction'].mean()

print(f"Mean Absolute Error (Home): {mae_home:.2f}")
print(f"Mean Absolute Error (Away): {mae_away:.2f}")
print(f"Winner Prediction Accuracy: {accuracy:.2%}")

merged[['season', 'week', 'homeTeam', 'awayTeam', 'homePoints', 'awayPoints', 'pred_homePoints', 'pred_awayPoints', 'actual_winner', 'pred_winner', 'correct_prediction']].head()

Mean Absolute Error (Home): 10.11
Mean Absolute Error (Away): 10.20
Winner Prediction Accuracy: 68.00%


,season,week,homeTeam,awayTeam,homePoints,awayPoints,pred_homePoints,pred_awayPoints,actual_winner,pred_winner,correct_prediction
0,2025,4,Charlotte,Rice,17,28,12.62,18.22,away,away,True
1,2025,4,Oklahoma State,Tulsa,12,19,26.58,14.97,away,home,False
2,2025,4,Rutgers,Iowa,28,38,28.86,24.20,away,home,False
3,2025,4,Miami (OH),UNLV,38,41,23.96,30.18,away,away,True
4,2025,4,Louisville,Bowling Green,40,17,29.32,12.71,home,home,True
